In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("data_ben2.csv")

In [11]:
data.head(3)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,fyr,...,dltt,dp,lct,ppegt,rect,sale,txp,xsga,cik,costat
0,1003,1990-01-31,1989,INDL,C,D,STD,A.A. IMPORTING CO INC,USD,1,...,0.076,0.790,10.449,2.281,1.373,19.267,0.000,8.916,730052.0,I
1,1004,1990-05-31,1989,INDL,C,D,STD,AAR CORP,USD,5,...,72.329,7.181,90.644,105.646,93.581,444.875,3.040,53.912,1750.0,A
2,1004,1991-05-31,1990,INDL,C,D,STD,AAR CORP,USD,5,...,68.953,8.256,79.227,113.060,91.238,466.542,4.019,58.545,1750.0,A


#### Deriving Features for beneish model

In [15]:
col = ['dsri','gmi','aqi','sgi','depi','sgai','lvgi','tata']

In [16]:
for c in col:
    data[c] = None

In [17]:
data.isna().sum()

gvkey            0
datadate         0
fyear            0
indfmt           0
consol           0
popsrc           0
datafmt          0
conm             0
curcd            0
fyr              0
act         107732
am          143703
at           58626
ch           63086
cogs         60135
dd1          73212
dltt         59512
dp           69452
lct         106110
ppegt        93032
rect         60992
sale         60096
txp          87147
xsga        108667
cik          38469
costat           0
dsri        327375
gmi         327375
aqi         327375
sgi         327375
depi        327375
sgai        327375
lvgi        327375
tata        327375
dtype: int64

In [18]:
data['gvkey'].nunique()

30920

In [19]:
data = data.sort_values(by=['gvkey','datadate'])

In [20]:
from collections import Counter
for key in data['gvkey'].unique():
    data[data['gvkey']==key] = data[data['gvkey']==key].fillna(data[data['gvkey']==key].mean())

C:\Users\saite\AppData\Local\Temp\ipykernel_1304\1392432458.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[data['gvkey']==key] = data[data['gvkey']==key].fillna(data[data['gvkey']==key].mean())


##### Filling null values with the mean values of the company

In [21]:
data = data.replace(0,0.0001)

In [22]:
data.isna().sum()

gvkey            0
datadate         0
fyear            0
indfmt           0
consol           0
popsrc           0
datafmt          0
conm             0
curcd            0
fyr              0
act          91620
am           83596
at           43931
ch           44654
cogs         44055
dd1          46218
dltt         43976
dp           48009
lct          88451
ppegt        73231
rect         44317
sale         44055
txp          69392
xsga         76843
cik          38469
costat           0
dsri        327375
gmi         327375
aqi         327375
sgi         327375
depi        327375
sgai        327375
lvgi        327375
tata        327375
dtype: int64

In [23]:
def compute_scores(key):
    df = data[data['gvkey']==key]
    data.loc[data['gvkey']==key,'dsri'] = ((df['rect']/df['sale'])/(df['rect'].shift(1)/df['sale'].shift(1))).values
    data.loc[data['gvkey']==key,'gmi'] = ((df['sale'].shift(1) -df['cogs'].shift(1))/df['sale'].shift(1))/((df['sale']-df['cogs'])/df['sale'])
    data.loc[data['gvkey']==key,'aqi'] = ((1-df['act']+df['ppegt'])/df['at'])/((1-df['act'].shift(1)+df['ppegt'].shift(1))/df['at'].shift(1))
    data.loc[data['gvkey']==key,'sgi'] = df['sale'] / df['sale'].shift(1)
    data.loc[data['gvkey']==key,'depi'] = ((df['dp'].shift(1) - df['am'].shift(1)) / ((df['dp'].shift(1) - df['am'].shift(1)) + df['ppegt'].shift(1)))/((df['dp'] - df['am']) / ((df['dp'] - df['am']) + df['ppegt']))
    data.loc[data['gvkey']==key,'sgai'] = (df['xsga']/df['sale'])/(df['xsga'].shift(1) /df['sale'].shift(1))
    data.loc[data['gvkey']==key,'lvgi']= ((df['dltt'] + df['lct']) / df['at']) / ((df['dltt'].shift(1) + df['lct'].shift(1)) / df['at'].shift(1))
    data.loc[data['gvkey']==key,'tata']= (df['act']-df['ch']-df['lct']-df['dd1']-df['txp']-df['dp'])/df['at']
    

In [24]:
for key in data['gvkey'].unique():
    compute_scores(key)


In [26]:
data.isna().sum()

gvkey            0
datadate         0
fyear            0
indfmt           0
consol           0
popsrc           0
datafmt          0
conm             0
curcd            0
fyr              0
act          91620
am           83596
at           43931
ch           44654
cogs         44055
dd1          46218
dltt         43976
dp           48009
lct          88451
ppegt        73231
rect         44317
sale         44055
txp          69392
xsga         76843
cik          38469
costat           0
dsri         69835
gmi          78426
aqi         113533
sgi          69528
depi        118463
sgai         99394
lvgi        110196
tata         93079
dtype: int64

In [27]:
data['m_score'] =  -4.840 + 0.920*data['dsri'] + 0.528*data['gmi'] + 0.404*data['aqi'] + 0.892*data['sgi'] + 0.115*data['depi'] - 0.172*data['sgai']
+ 4.679*data['tata'] - 0.327*data['lvgi']

0              NaN
1              NaN
2         1.835555
3         1.801839
4         2.011009
            ...   
327370   -0.635386
327371         NaN
327372   -1.281769
327373         NaN
327374    0.061737
Length: 327375, dtype: object

In [28]:
data = data.dropna(subset=['m_score'])

In [29]:
import numpy as np
def find_fraud(x):
    if np.isnan(x):
        return None
    if x > -1.78:
        return 0
    else:
        return 1

In [30]:
data['fraud'] = data['m_score'].apply(lambda x: find_fraud(x))

In [31]:
data

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,fyr,...,dsri,gmi,aqi,sgi,depi,sgai,lvgi,tata,m_score,fraud
2,1004,1991-05-31,1990,INDL,C,D,STD,AAR CORP,USD,5,...,0.929684,1.126358,0.934217,1.048704,0.924128,1.035504,0.929721,0.457272,-2.148938,1
3,1004,1992-05-31,1991,INDL,C,D,STD,AAR CORP,USD,5,...,1.086204,0.993967,1.053741,0.905936,1.061972,1.073004,1.035224,0.457439,-2.144502,1
4,1004,1993-05-31,1992,INDL,C,D,STD,AAR CORP,USD,5,...,0.846744,1.101064,0.930615,0.905652,1.17106,1.010717,0.935651,0.495184,-2.334995,1
5,1004,1994-05-31,1993,INDL,C,D,STD,AAR CORP,USD,5,...,1.171883,1.013483,1.109818,1.065244,0.976298,0.902585,1.162818,0.502483,-1.871156,1
6,1004,1995-05-31,1994,INDL,C,D,STD,AAR CORP,USD,5,...,1.160536,0.993977,1.02226,1.107028,0.746802,0.963684,1.031365,0.492955,-1.926897,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327368,351590,2020-12-31,2020,INDL,C,D,STD,DAIMLER TRUCK HOLDING AG,USD,12,...,0.997366,1.084907,-0.477734,0.836501,0.949486,1.09888,0.982883,-0.234252,-2.876255,1
327369,351590,2021-12-31,2021,INDL,C,D,STD,DAIMLER TRUCK HOLDING AG,USD,12,...,0.968646,0.94129,-2.996143,1.041812,1.215784,0.942659,0.862599,-0.075748,-3.755312,1
327370,351590,2022-12-31,2022,INDL,C,D,STD,DAIMLER TRUCK HOLDING AG,USD,12,...,1.029327,0.995529,2.081224,1.201161,0.992461,0.877754,1.032458,-0.06364,-1.491971,0
327372,353444,2022-12-31,2022,INDL,C,D,STD,HALEON PLC,USD,12,...,0.43,1.015307,0.382588,1.012218,1.076264,0.968241,3.288491,-0.044119,-2.893621,1


In [32]:
data.to_csv("Beneish_scores.csv")

In [33]:
data2 = pd.read_csv("bao_all_data.csv")

In [39]:
len(data)

176915

In [45]:
dets = pd.merge(data,data2[['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt','Mistate']],on=['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt'],how='inner')

In [52]:
dets.to_csv("Beneish_scores_final.csv")

In [53]:
from sklearn.metrics import roc_auc_score
import numpy as np



auc = roc_auc_score(dets['Mistate'], dets['fraud'])


print("AUC:", auc)


AUC: 0.4820546137857691


### Probit Model

In [143]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [144]:
X = dets[['dsri','gmi','aqi','sgi','depi','sgai','lvgi','tata','Mistate']]

In [145]:
X = X.replace([np.inf,-np.inf],np.nan)

In [146]:
X = X.dropna()

In [147]:
for col in X.columns:
    X[col] = X[col].astype(float)

In [148]:
X = sm.add_constant(X)

In [149]:
X

,const,dsri,gmi,aqi,sgi,depi,sgai,lvgi,tata,Mistate
0,1.0,0.929684,1.126358,0.934217,1.048704,0.924128,1.035504,0.929721,0.457272,0.0
1,1.0,1.086204,0.993967,1.053741,0.905936,1.061972,1.073004,1.035224,0.457439,0.0
2,1.0,0.846744,1.101064,0.930615,0.905652,1.171060,1.010717,0.935651,0.495184,0.0
3,1.0,1.171883,1.013483,1.109818,1.065244,0.976298,0.902585,1.162818,0.502483,0.0
4,1.0,1.160536,0.993977,1.022260,1.107028,0.746802,0.963684,1.031365,0.492955,0.0
...,...,...,...,...,...,...,...,...,...,...
176473,1.0,0.628779,1.032892,0.561158,1.066417,1.034979,0.963655,1.155294,-0.530692,0.0
176474,1.0,0.997366,1.084907,-0.477734,0.836501,0.949486,1.098880,0.982883,-0.234252,0.0
176475,1.0,0.968646,0.941290,-2.996143,1.041812,1.215784,0.942659,0.862599,-0.075748,0.0
176476,1.0,1.029327,0.995529,2.081224,1.201161,0.992461,0.877754,1.032458,-0.063640,0.0


In [150]:
y= X['Mistate']
X = X.drop(['Mistate'],axis=1) 

In [152]:
model = sm.Probit(y, X)

In [153]:
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.033575
         Iterations 19
                          Probit Regression Results                           
Dep. Variable:                Mistate   No. Observations:               173596
Model:                         Probit   Df Residuals:                   173587
Method:                           MLE   Df Model:                            8
Date:                Tue, 12 Sep 2023   Pseudo R-squ.:                0.001573
Time:                        20:41:54   Log-Likelihood:                -5828.5
converged:                       True   LL-Null:                       -5837.7
Covariance Type:            nonrobust   LLR p-value:                   0.01864
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.5386      0.014   -178.623      0.000      -2.566      -2.511
dsri       -5.389e-08   3.09